In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Определяем класс набора данных
class FurnitureDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        image = Image.open(self.image_paths[index])
        label = self.labels[index]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Определяем пути к изображениям из набора данных и соответствующим меткам
image_paths = ['/path/to/image1.jpg', '/path/to/image2.jpg', ...]
labels = ['table', 'chair', ...]

# Определяем преобразование, которое будет применяться к изображениям
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Создаём набор данных
dataset = FurnitureDataset(image_paths, labels, transform)

# Создаём загрузчик данных
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Определяем архитектуру модели для классификации (например, на основе ResNet)
model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
num_classes = len(set(labels))
model.fc = torch.nn.Linear(512, num_classes)

# Определяем функцию потерь и оптимизатор
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Обучаем модель
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(10):
    running_loss = 0.0
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    print(f"Epoch {epoch+1} Loss: {running_loss}")

# Тестируем модель на реальных изображениях
test_image = Image.open('/path/to/test_image.jpg')
test_image = transform(test_image).unsqueeze(0).to(device)  # Добавить пакетное измерение и переместить на устройство
output = model(test_image)
predicted_class = torch.argmax(output).item()

# Показать предсказанный класс
print(f"Predicted class: {predicted_class}")